In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

/home/aeon/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/aeon/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
data_path = "data/"
#Load OECD Better Life Index data.
bli_df = pd.read_csv(data_path + "oecd_bli_2015.csv")
#Load IMF GDP per capita data.
#Encoding is different from Unicode, so Unicode based decoding won't work.
#Have to use latin1
gdp_df = pd.read_csv(data_path + "gdp_per_capita.csv", delimiter="\t", encoding="latin1")

In [11]:
#See the BLI data.
bli_df.head(2)

,LOCATION,Country,INDICATOR,Indicator,MEASURE,Measure,INEQUALITY,Inequality,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,HO_BASE,Dwellings without basic facilities,L,Value,TOT,Total,PC,Percentage,0,units,NaN,NaN,1.1,E,Estimated value
1,AUT,Austria,HO_BASE,Dwellings without basic facilities,L,Value,TOT,Total,PC,Percentage,0,units,NaN,NaN,1.0,NaN,NaN


In [12]:
#See the GDP data.
gdp_df.head(2)

,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2015,Estimates Start After
0,Afghanistan,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",599.994,2013.0
1,Albania,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...","3,995.383",2010.0


In [34]:
#Check if 'Country' field qualifies as a primary key for join.
print("Country field in BLI Table is a Primary Key : ", ~bli_df["Country"].duplicated().any())
print("Country field in GDP Table is a Primary Key : ", ~gdp_df["Country"].duplicated().any())

Country field in BLI Table is a Primary Key :  False
Country field in GDP Table is a Primary Key :  True


In [28]:
#Check why 'Country' is not a proper primary key.
bli_df["Country"].value_counts()

Slovak Republic    90
Russia             90
Portugal           90
Switzerland        90
Chile              90
Hungary            90
New Zealand        90
Japan              90
Ireland            90
Greece             90
Norway             90
Sweden             90
Mexico             90
Finland            90
Iceland            90
Korea              90
Czech Republic     90
Turkey             90
Poland             90
Estonia            90
Italy              90
Spain              89
United Kingdom     89
Luxembourg         89
Austria            89
Canada             89
France             89
Australia          89
Germany            89
Netherlands        88
Brazil             88
Slovenia           88
United States      88
Belgium            88
Israel             88
Denmark            88
OECD - Total       74
Name: Country, dtype: int64

In [29]:
#Check why multiple entries against same "Country" value.
bli_df[bli_df["Country"] == "Australia"]

,LOCATION,Country,INDICATOR,Indicator,MEASURE,Measure,INEQUALITY,Inequality,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,HO_BASE,Dwellings without basic facilities,L,Value,TOT,Total,PC,Percentage,0,units,NaN,NaN,1.100000,E,Estimated value
37,AUS,Australia,HO_BASE,Dwellings without basic facilities,L,Value,MN,Men,PC,Percentage,0,units,NaN,NaN,1.100000,E,Estimated value
73,AUS,Australia,HO_BASE,Dwellings without basic facilities,L,Value,WMN,Women,PC,Percentage,0,units,NaN,NaN,1.100000,E,Estimated value
109,AUS,Australia,HO_HISH,Housing expenditure,L,Value,TOT,Total,PC,Percentage,0,units,NaN,NaN,20.000000,NaN,NaN
146,AUS,Australia,HO_HISH,Housing expenditure,L,Value,MN,Men,PC,Percentage,0,units,NaN,NaN,20.000000,E,Estimated value
182,AUS,Australia,HO_HISH,Housing expenditure,L,Value,WMN,Women,PC,Percentage,0,units,NaN,NaN,20.000000,E,Estimated value
218,AUS,Australia,HO_NUMR,Rooms per person,L,Value,TOT,Total,RATIO,Ratio,0,units,NaN,NaN,2.300000,E,Estimated value
255,AUS,Australia,HO_NUMR,Rooms per person,L,Value,MN,Men,RATIO,Ratio,0,units,NaN,NaN,2.300000,E,Estimated value
291,AUS,Australia,HO_NUMR,Rooms per person,L,Value,WMN,Women,RATIO,Ratio,0,units,NaN,NaN,2.300000,E,Estimated value
327,AUS,Australia,IW_HADI,Household net adjusted disposable income,L,Value,TOT,Total,USD,US Dollar,0,units,NaN,NaN,31588.000000,NaN,NaN


In [35]:
#Seems like there are multiple indicator values in the BLI dataset.
#Life satisfaction for each country is most probably uniquely defined by 
#INDICATOR="SW_LIFS" and INEQUALITY="TOT".
#Create an appropriate filter on these.
bli_filter = (bli_df["INDICATOR"] == "SW_LIFS") & (bli_df["INEQUALITY"] == "TOT")
#Check if filter applied BLI table has 'Country' field as valid primary key.
print("Filter applied BLI Table is a Primary Key : ", ~bli_df[bli_filter]["Country"].duplicated().any())

Filter applied BLI Table is a Primary Key :  True


In [36]:
#Select only relevant data before Join operation.
bli_df = bli_df[bli_filter]

In [37]:
#Do an Inner Join both Dataframes on 'Country' field and select relavent columns.
bli_gdp_df = pd.merge(bli_df, gdp_df, on="Country", how="inner")[["Country", "Value", "2015"]]
bli_gdp_df.head(2)

,Country,Value,2015
0,Australia,7.3,"50,961.865"
1,Austria,6.9,"43,724.031"


In [38]:
#Rename the column names to appropriately.
bli_gdp_df.rename(columns={"Value":"Better Life Index", "2015":"GDP per capita"}, inplace=True)
bli_gdp_df.head(2)

,Country,Better Life Index,GDP per capita
0,Australia,7.3,"50,961.865"
1,Austria,6.9,"43,724.031"
